# Tugas 1 : Basic NLP Tools
Oleh : 
- Safiq Faray (13519145)

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
text = nlp("Hello, i live in Indonesia and i love lasagna. Also, i love burgers and pizzas. Man, i wish i could get some pizza right now.")

## Sentence Splitter

In [3]:
def sentence_split(text):
    return list(text.sents)
sentence_split(text)

[Hello, i live in Indonesia and i love lasagna.,
 Also, i love burgers and pizzas.,
 Man, i wish i could some pizza right now.]

## Tokenization

In [4]:
def tokenization(text):
    return [token.text for token in text]
tokenization(text)

['Hello',
 ',',
 'i',
 'live',
 'in',
 'Indonesia',
 'and',
 'i',
 'love',
 'lasagna',
 '.',
 'Also',
 ',',
 'i',
 'love',
 'burgers',
 'and',
 'pizzas',
 '.',
 'Man',
 ',',
 'i',
 'wish',
 'i',
 'could',
 'some',
 'pizza',
 'right',
 'now',
 '.']

## Stemming

In [7]:
from nltk.stem import PorterStemmer
def stemming(token):
    stemmer = PorterStemmer()
    return stemmer.stem(token.text)
tes = nlp("writing")
stemming(tes[0])

'write'